In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn

In [2]:
folder_path = '/content/drive/MyDrive/python_data/kaggle/instacart/data/'
files = ['aisles.csv', 'departments.csv', 'order_products__prior.csv', 'order_products__train.csv', 'orders.csv', 'products.csv']

In [3]:
# aisle = pd.read_csv(folder_path+files[0])
# dep = pd.read_csv(folder_path+files[1])
prior = pd.read_csv(folder_path+files[2])
train = pd.read_csv(folder_path+files[3])
orders = pd.read_csv(folder_path+files[4])
products = pd.read_csv(folder_path+files[5])
test = orders[orders['eval_set'] == 'test']

In [4]:
print(len(products))
products.head(2)

49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13


In [5]:
orders.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [6]:
prior.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1


In [7]:
print(len(train))
train.head(2)

1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1


In [8]:
# orders[orders['user_id']==1]

In [9]:
# train[train['order_id']==1187899]

# Preprocessing
* 將prior資訊 agg: df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})

## Merging 

In [10]:
train_order = orders[orders['eval_set']=='train'].drop(columns= ['eval_set'], axis= 1)
prior_order = orders[orders['eval_set']=='prior'].drop(columns= ['eval_set'], axis= 1)
train_order.columns = ['train_'+col for col in train_order.columns]
prior_order.columns = ['prior_'+col for col in prior_order.columns]

In [11]:
train_prior = pd.merge(train_order, prior_order, left_on= 'train_user_id', right_on='prior_user_id')
train_prior.head(2)

,train_order_id,train_user_id,train_order_number,train_order_dow,train_order_hour_of_day,train_days_since_prior_order,prior_order_id,prior_user_id,prior_order_number,prior_order_dow,prior_order_hour_of_day,prior_days_since_prior_order
0,1187899,1,11,4,8,14.0,2539329,1,1,2,8,NaN
1,1187899,1,11,4,8,14.0,2398795,1,2,3,7,15.0


In [12]:
train_X = pd.merge(train_prior, prior, left_on= 'prior_order_id', right_on= 'order_id')
train_X = pd.merge(train_X, products, left_on= 'product_id', right_on= 'product_id')

## 觀察
* 對於同個train order id且相同product，可能會有許多不同prior order id的訊息，即相同的商品可能買過很多次，要如何組成一組feature？
* 平均每個product的prior order數: 2.44
* 最大每個product的prior order數: 99

In [13]:
print(len(train_X))
train_X.head(3)

20641991


,train_order_id,train_user_id,train_order_number,train_order_dow,train_order_hour_of_day,train_days_since_prior_order,prior_order_id,prior_user_id,prior_order_number,prior_order_dow,prior_order_hour_of_day,prior_days_since_prior_order,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1187899,1,11,4,8,14.0,2539329,1,1,2,8,NaN,2539329,196,1,0,Soda,77,7
1,1187899,1,11,4,8,14.0,2398795,1,2,3,7,15.0,2398795,196,1,1,Soda,77,7
2,1187899,1,11,4,8,14.0,473747,1,3,3,12,21.0,473747,196,1,1,Soda,77,7


In [14]:
avg_n_prior_order = np.mean(train_X.groupby(['train_order_id', 'product_id'])['prior_order_id'].count())
max_n_prior_order = np.max(train_X.groupby(['train_order_id', 'product_id'])['prior_order_id'].count())
print(f'平均每個product的prior order數: {round(avg_n_prior_order, 2)}')
print(f'最大每個product的prior order數: {round(max_n_prior_order, 2)}')

平均每個product的prior order數: 2.44
最大每個product的prior order數: 99


In [15]:
cols = ['train_order_id', 'train_order_dow', 'train_order_hour_of_day', 'train_days_since_prior_order', 'product_id', 'aisle_id', 'department_id']
# train_X.groupby(cols).agg({'prior_order_dow':lambda x: x.mode()[0], 'order_id':'count'})
grouped_train_X = train_X.groupby(cols).agg({'prior_order_id':'count'}).add_suffix('_Count').reset_index()
# train_X.groupby(cols).count().reset_index()

In [16]:
print(grouped_train_X)
grouped_train_X.head(3)

,train_order_id,train_order_dow,train_order_hour_of_day,train_days_since_prior_order,product_id,aisle_id,department_id,order_id_Count
0,1,4,10,9.0,2067,3,19,1
1,1,4,10,9.0,5707,3,19,2
2,1,4,10,9.0,11109,108,16,2


## 產生Y [0, 1]

In [17]:
# train_X['label'] = 0 

In [23]:
train_array= np.array(train[['order_id', 'product_id']])
grouped_train_array= np.array(grouped_train_X[['train_order_id', 'product_id']])

In [ ]:
y= [1 if row in train_array else 0 for row in grouped_train_array]

In [20]:
# for i, row in train.iterrows():
#   if row['reordered'] == 1:
#     train_X.loc[(train_X['train_order_id']==row['order_id'])&(train_X['product_id']==row['product_id']), 'label'] = 1